### Exploring Relationship between Temperature Change and Health Expenditure Globally

##### Cole Smith,Ginger Hudson, Ritika Palacharla

In [ ]:
# import libraries
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import altair as alt
import numpy as np
import plotly.io as pio
import json

#### Data Cleaning and Standardization

In [ ]:
# import simplified dataset from World Health Organization
df_health = pd.read_csv('Simplified_GHED_data.csv')

In [ ]:
df_health.info()

In [ ]:
df_health.shape

In [ ]:
df_health.head(5)

In [ ]:
df_health.tail(5)

In [ ]:
# delete 'Unnamed' column because it is the same as index
df_health = df_health.drop('Unnamed: 0', axis = 1)

##### Via GHED codebook:
- Income: Country's income group for 2021 via Worldbank
- che: Current health expenditure in millions of respective National Currency Unit (NCU)
- che_gdp: Current health expenditure as a percentage of country GDP
- che_pc_usd: Current health expenditure per capita in USD
- gghed: Domestic general governmental health expenditure in millions NCU
- pvtd: Domestic private health expenditure in millions NCU
- pop: Population in thousands

In [ ]:
df_health.columns

In [ ]:
# classify the columns as categorical, ordinal, discrete, continuous
grouped_columns = df_health.columns.to_series().groupby(df_health.dtypes).groups
classifications = {}
dtype_classifications = {
    'object': 'categorical',
    'int64' : 'discrete',
    'float64' : 'continous'
}
for dtype, columns in grouped_columns.items():
  for column in columns:
    classifications[column] = dtype_classifications[str(dtype)]
# print out the classifications stored in in classifications{}
for column, classification in classifications.items():
  print(f"'{column}' is '{classification}'")

In [ ]:
# Find discrete (unique) values for 'country', 'income', 'region'
df_health['country'].unique()

In [ ]:
df_health['income'].unique()

In [ ]:
df_health['region'].unique()

In [ ]:
df_health[['country']].value_counts()

In [ ]:
df_health[['income']].value_counts()

In [ ]:
df_health[['region']].value_counts()

In [ ]:
# Find mode of each categorical column
df_health['country'].mode()

In [ ]:
df_health['income'].mode()

In [ ]:
df_health['region'].mode()

In [ ]:
# create dictionary for classification
columns_classification = {
    'year': 'discrete',
    'che_gdp': 'continuous',
    'che_pc_usd': 'continuous',
    'che': 'continuous',
    'gghed': 'continuous',
    'pvtd': 'continuous',
    'pop': 'continuous'
}
# empty dictionary to store statistics
statistics = {}

for column, classification in columns_classification.items():
    stats = {
        'Range': df_health[column].max() - df_health[column].min(),
        'Mean': df_health[column].mean(),
        'Median': df_health[column].median(),
        'Standard Deviation': df_health[column].std(),
    }
    statistics[column] = stats

# the statistics for each column
for column, stats in statistics.items():
    print(f"Statistics for column '{column}':")
    for stat_name, value in stats.items():
        print(f"\t{stat_name}: {value}")

In [ ]:
# find missing data
missing_data = df_health.isnull()

In [ ]:
# count missing data in each column
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

In [ ]:
# replace missing che_gdp data with mean
df_health['che_gdp'].replace(np.nan,df_health['che_gdp'].mean(), inplace = True)

In [ ]:
# replace missing che_pc_usd data with mean
df_health['che_pc_usd'].replace(np.nan,df_health['che_pc_usd'].mean(), inplace = True)

In [ ]:
# replace missing che data with mean
df_health['che'].replace(np.nan,df_health['che'].mean(), inplace = True)

In [ ]:
# replace missing gghed data with mean
df_health['gghed'].replace(np.nan,df_health['gghed'].mean(), inplace = True)

In [ ]:
# replace missing pvtd data with mean
df_health['pvtd'].replace(np.nan,df_health['pvtd'].mean(), inplace = True)

In [ ]:
# replace missing pop data with mean
df_health['pop'].replace(np.nan,df_health['pop'].mean(), inplace = True)

In [ ]:
# confirm missing data has been replaced
df_health.info()

In [ ]:
# check for duplicates
# run through duplicates test
# if duplicates - drop the rows
pd.options.display.max_rows = 100
print(df_health.duplicated())

In [ ]:
# drop duplicates
df_nodup = df_health.drop_duplicates()

In [ ]:
df_health.shape

In [ ]:
# Confirming no duplicate rows by matching num rows
df_nodup.shape

#### Correlations and Plotting Relationships

In [ ]:
# determining correlation for GHED data
df_health.corr(method = 'pearson', numeric_only = True)

In [ ]:
# import linear regression model and seaborn
from sklearn.linear_model import LinearRegression
import seaborn as sns

# determine best fit for high correlations
# pvtd and gghed
# gghed and che
# pvtd and che

In [ ]:
# create linear regression object
lm = LinearRegression()

# label variables to check
X = df_health[['pvtd']]
Y = df_health['gghed']

# fit linear model
lm.fit(X,Y)

In [ ]:
# plot linear regression
sns.regplot(
    data = df_health,
    x = 'pvtd',
    y = 'gghed'
)
plt.ylim(0,)
plt.show()

In [ ]:
# repeat with gghed and che

# label variables to check
X = df_health[['gghed']]
Y = df_health['che']

# fit linear model
lm.fit(X,Y)

In [ ]:
# plot linear regression
sns.regplot(
    data = df_health,
    x = 'che',
    y = 'gghed'
)
plt.ylim(0,)
plt.show()

In [ ]:
# repeat with pvtd and che

# label variables to check
X = df_health[['che']]
Y = df_health['pvtd']

# fit linear model
lm.fit(X,Y)

In [ ]:
# plot linear regression with che and pvtd
sns.regplot(
    data = df_health,
    x = 'che',
    y = 'pvtd'
)
plt.ylim(0,)
plt.show()

#### Analysis for global city temperature data

In [ ]:
# read in global city_temperature data from Berkeley Earth dataset
df_temp = pd.read_csv('city_temperature.csv', dtype = {
                    'State': 'object', 'Month': int, 'Day': int, 'Year': int, 'AvgTemperature': float}
)

In [ ]:
df_temp.shape

In [ ]:
df_temp.info()

In [ ]:
df_temp.head(5)

In [ ]:
df_temp.tail(5)

In [ ]:
# Via data documentation:
# State: 50% missing data as only applicable to certain countries
# AvgTemperature: in degrees Fahrenheit
df_temp.columns

In [ ]:
# classify datatypes

column_classification = {
    'Region': 'Categorical',
    'Country': 'Categorical',
    'State': 'Categorical',
    'City': 'Categorical',
    'Month': 'Ordinal',
    'Day': 'Ordinal',
    'Year': 'Ordinal',
    'AvgTemperature': 'Continuous'
}

for column, classification in column_classification.items():
    print(f"{column}: {classification}")

In [ ]:
# Find discrete values for 'Region', 'Country', 'State', 'City'
df_temp['Region'].unique()

In [ ]:
df_temp['Country'].unique()

In [ ]:
df_temp['State'].unique()

In [ ]:
df_temp['City'].unique()

In [ ]:
# Find frequency distribution for categorical columns
df_temp[['Region']].value_counts()

In [ ]:
df_temp[['Country']].value_counts()

In [ ]:
df_temp[['State']].value_counts()

In [ ]:
df_temp[['City']].value_counts()

In [ ]:
# Find mode for categorical columns
df_temp['Region'].mode()

In [ ]:
df_temp['Country'].mode()

In [ ]:
df_temp['State'].mode()

In [ ]:
df_temp['City'].mode()

In [ ]:
# get basic statistics for non-categorical columns
temp_columns_classification = {
    'Month' : 'ordinal',
    'Day' : 'ordinal',
    'Year' : 'ordinal',
    'AvgTemperature' : 'continuous'
}

# empty dictionary to store statistics
temp_statistics = {}

for column, classification in temp_columns_classification.items():
    stats = {
        'Range': df_temp[column].max() - df_temp[column].min(),
        'Mean': df_temp[column].mean(),
        'Median': df_temp[column].median(),
        'Standard Deviation': df_temp[column].std(),
    }
    temp_statistics[column] = stats

# the statistics for each column
for column, stats in temp_statistics.items():
    print(f"Statistics for column '{column}':")
    for stat_name, value in stats.items():
        print(f"\t{stat_name}: {value}")

In [ ]:
# find missing data
missing_data = df_temp.isnull()

In [ ]:
# count missing data in each column
# missing data in State column cannot be replaced
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print('')

In [ ]:
# check for duplicates
# run through duplicates test
# if duplicates - drop the rows
pd.options.display.max_rows = 2000000
print(df_temp.duplicated())

In [ ]:
# drop duplicates
df_nodup = df_temp.drop_duplicates()

In [ ]:
df_temp.shape

In [ ]:
# about 10,000 duplicates to be deleted
df_nodup.shape

In [ ]:
# show correlation of 2nd dataset
# there is no correlation which tracks for the dataset which follows from type of data
df_temp.corr(method = 'pearson', numeric_only = True)

In [ ]:
# null temp value sometimes shown as -99.0 therefore clearing null rows
df_temp.drop(df_temp[df_temp['AvgTemperature'] == -99.0].index, inplace = True)

#### Merging Dataframes

In [ ]:
# if value in df_health and not in df_temp then wil be dropped in inner join
# replace country names with more basic name for more merge results
df_health.replace('Lao People\'s Democratic Republic','Laos')
df_health.replace('Netherlands (Kingdom of the)','The Netherlands')
df_health.replace('Russian Federation','Russia')
df_health.replace(['Serbia','Montenegro'],'Serbia-Montenegro')
df_health.replace('United Kingdom of Great Britain and Northern Ireland','United Kingdom')
df_health.replace('Bolivia (Plurinational State of)','Bolivia')
df_health.replace('Venezuela (Bolivarian Republic of)','Venezuela')
df_health.replace('United States of America','US')
df_health.replace('Côte d\'Ivoire','Ivory Coast')
df_health.replace('Türkiye','Turkey')
df_health.replace('Republic of Korea','South Korea')
df_health.replace('Viet Nam','Vietnam')
df_temp.replace('Myanmar (Burma)','Myanmar')
df_temp.replace('Czech Republic','Czechia')
df_health.replace('United Republic of Tanzania','Tanzania')
df_health.replace('Syrian Arab Republic','Syria')
df_health.replace('North Macedonia','Macedonia')
df_temp.replace('Equador','Ecuador')

In [ ]:
# create group with wanted data and group by country and year
# in preparation for merge
df_temp_group = df_temp[['Country','Year','AvgTemperature']]

df_temp_group = df_temp_group.groupby(['Country','Year']).mean()

In [ ]:
df_health.rename(columns = {'country':'Country',
                            'year':'Year'},
                 inplace = True)
df_health.head()

In [ ]:
merged_df = df_temp_group.merge(df_health,
                how = 'inner',
                on = ['Country','Year']
               )
merged_df.head(200)

In [ ]:
merged_df.shape

In [ ]:
# check for correlation of merged_df
merged_df.corr(numeric_only= 'True')

In [ ]:
# merged df currency standardization
merged_df['che_usd_per_capita'] = (merged_df['che'] * merged_df['che_pc_usd']) / merged_df['pop']
merged_df['pvtd_usd_per_capita'] = (merged_df['pvtd'] * merged_df['che_pc_usd']) / merged_df['pop']
merged_df['gghed_usd_per_capita'] = (merged_df['gghed'] * merged_df['che_pc_usd']) / merged_df['pop']

merged_df.head()

### Plotting Trends and Creating Visualizations

#### Plotly Chloropleths

In [ ]:
# generating average temperatures by country and year for chloropleth
annual_avg_temp = df_temp.groupby(['Country', 'Year'])['AvgTemperature'].mean().reset_index()
global_min_temp = annual_avg_temp['AvgTemperature'].min()
global_max_temp = annual_avg_temp['AvgTemperature'].max()

In [ ]:
# sorting as integer
annual_avg_temp['Year'] = annual_avg_temp['Year'].astype(int)
annual_avg_temp = annual_avg_temp.sort_values('Year')

In [ ]:
# plotting chloropleth map
fig = px.choropleth(
    annual_avg_temp,
    locations='Country',
    locationmode='country names',
    color='AvgTemperature',
    hover_name='Country',
    animation_frame='Year',
    color_continuous_scale=px.colors.sequential.Jet,
    title='Annual Average Temperature by Country',
    range_color=[global_min_temp, global_max_temp]
)

fig.update_layout(
    title_text = 'Annual Average Temperature in Fahrenheit over Time',
    width = 800,
    height = 500,
    geo = dict(projection = {'type' : 'hammer'})
)

fig.show()

plot_json = pio.to_json(fig)
with open('plot_data.json', 'w') as f:
    json.dump(plot_json, f)

In [ ]:
# create inflation adjusted column using U.S. Urban Consumers CPI Feb. 2024
df2 = df_health
df2 = df2.drop(df2[df2['Year'] > 2020].index)
df2['che_pc_usd_infl'] = (df2['che_pc_usd'] / 605.162) * 100

In [ ]:
# define max and min for range in plot
che_pc_max = df2['che_pc_usd_infl'].max()
che_pc_min = df2['che_pc_usd_infl'].min()

In [ ]:
# create choropleth for current health expenditure per capita in USD
fig = px.choropleth(
    df2,
    locations = 'Country',
    locationmode = 'country names',
    color = 'che_pc_usd_infl',
    hover_name = 'Country',
    animation_frame = 'Year',
    color_continuous_scale = px.colors.sequential.Sunsetdark,
    title = 'Inflation Adjusted Health Expenditure Per Capita',
    range_color = [che_pc_min,che_pc_max]
)

fig.update_layout(
    width = 800,
    height = 700,
    geo = dict(projection = {'type' : 'hammer'})
)

fig.show()

plot_json2 = pio.to_json(fig)
with open('plot_data2.json', 'w') as f:
    json.dump(plot_json2, f)

#### Altair Scatterplot

In [ ]:
# Calculating percent change for 'che_usd_per_capita' and average temperature from 2000 to 2020
merged_df_2000 = merged_df[merged_df['Year'] == 2000].set_index('Country')
merged_df_2020 = merged_df[merged_df['Year'] == 2020].set_index('Country')

merged_df_diff = merged_df_2000.join(merged_df_2020, lsuffix='_2000', rsuffix='_2020')

merged_df_diff['che_usd_per_capita_percent_change'] = ((merged_df_diff['che_usd_per_capita_2020'] - merged_df_diff['che_usd_per_capita_2000']) / merged_df_diff['che_usd_per_capita_2000']) * 100
merged_df_diff['che_usd_per_capita_percent_change'].fillna(0, inplace=True)

merged_df_diff['che_usd_per_capita_percent_change'] = merged_df_diff['che_usd_per_capita_percent_change'].round(2)

avg_temperature_2000 = merged_df_2000.groupby('Country')['AvgTemperature'].mean().reset_index()
avg_temperature_2000.columns = ['Country', 'AvgTemperature_mean_2000']

avg_temperature_2020 = merged_df_2020.groupby('Country')['AvgTemperature'].mean().reset_index()
avg_temperature_2020.columns = ['Country', 'AvgTemperature_mean_2020']

merged_avg_temperature = pd.merge(avg_temperature_2000, avg_temperature_2020, on='Country')
merged_avg_temperature['AvgTemperature_mean'] = (merged_avg_temperature['AvgTemperature_mean_2000'] + merged_avg_temperature['AvgTemperature_mean_2020']) / 2

combined_df = pd.merge(merged_df_diff, merged_avg_temperature, on='Country')


In [ ]:
# scatterplot for change in health expenditure vs country's avg temp
scatterplot = alt.Chart(combined_df).mark_circle(size=100).encode(  # Adjust the size as needed
    x=alt.X('AvgTemperature_mean', title='Average Temperature', scale=alt.Scale(domain=[20, 90])),
    y=alt.Y('che_usd_per_capita_percent_change', title='Percent Change in Healthcare Expenditure (USD)', scale=alt.Scale(domain=[0, 15000])),
    color=alt.Color('region_2000', title='Region'),
    tooltip=['Country:N']
).properties(
    width=600,
    height=400,
    title = 'Change in Health Expenditure vs. Average Temperature'
).interactive()

scatterplot

In [ ]:
# Save the chart as HTML for dashboard
html_path = "scatterplot.html"
scatterplot.save(html_path)

#### Altair plot #2

In [ ]:
# graph to show difference in growth of private and domestic healthcare over time
base = alt.Chart(merged_df).encode(
    x=alt.X('Year:O', axis=alt.Axis(title='Year')),
    tooltip=['Country:N', 'Year:O', 'pop:Q']
).properties(
    width=800,
    height=600
).interactive()

private_healthcare = base.mark_circle(size=60, color='blue').encode(
    y=alt.Y('pvtd_usd_per_capita:Q', axis=alt.Axis(title='Private Healthcare Spending (USD)',
                                                   titleColor='blue'))
)

domestic_healthcare = base.mark_circle(size=60, color='green').encode(
    y=alt.Y('gghed_usd_per_capita:Q', axis=alt.Axis(title='Domestic Healthcare Spending (USD)',
                                                    titleColor='green'))
)

layered_chart = alt.layer(private_healthcare, domestic_healthcare).resolve_scale(
    y='independent'
)

layered_chart

In [ ]:
# Save the chart as HTML
html_path = "layered_chart.html"
layered_chart.save(html_path)

#### Altair Bar Chart and Linked Line Chart

In [ ]:
# graph to show health expenditure as percentage of GDP for different years
# linked to a selected country's temperature over time 
# utilizing selection_point filter for interactivity
merged_df = merged_df.sort_values(by=['che_gdp'])

year_selector = alt.selection_point(
    name = 'Year',
    fields = ['Year'],
    bind = alt.binding_range(min = 2000, max = 2020, step = 1, name = 'Select Year:'),
    value = 2000
)

che_year_chart = alt.Chart(merged_df).mark_bar().encode(
    alt.X('Country:N', sort=None).title('Country'),
    alt.Y('che_gdp:Q').scale(domain=(0,20)).title('Healthcare Expenditure as a % of GDP'),
    alt.Color('income:O').scale(domain=['High','Upper-middle','Lower-middle','Low'],
                                range=['#002253','#3A5888','#7095BF','#ADD5F7']),
    tooltip = ['Country','che_gdp']
).properties(
    width = 1000,
    title = 'Global Income, Healthcare Spending, and Temperature over Time'
).add_params(
    year_selector,
).transform_filter(
    year_selector
)

# graph to show relationship between average temperature and WHO's income level

country_temp = alt.Chart(merged_df).mark_line().encode(
    alt.X('Country'),
    alt.Y('AvgTemperature').title('Yearly Average Temperature (°F)'),
    alt.Color('income:O').scale(domain=['High','Upper-middle','Lower-middle','Low'],
                                range=['#002253','#3A5888','#7095BF','#ADD5F7'])
).transform_filter(
    year_selector
).properties(
    width = 1000,
    height = 200
)

che_year_chart & country_temp

In [ ]:
# save as html for dashboard
chart_to_save = che_year_chart & country_temp
html_path = "linebar.html"
chart_to_save.save(html_path)